---
Setting File Paths

---

In [1]:
# Input and output folders
file_path_leg_validation = '/Users/rishabhdubey/Downloads/Project/Feature Vectors/17th/features_dict_vision_val_leg_aug.pkl'
file_path_leg_test = '/Users/rishabhdubey/Downloads/Project/Feature Vectors/17th/features_dict_vision_test_leg_aug.pkl'
file_path_leg_train = '/Users/rishabhdubey/Downloads/Project/Feature Vectors/17th/features_dict_vision_leg_aug.pkl'
file_path_hand_validation = '/Users/rishabhdubey/Downloads/Project/Feature Vectors/17th/features_dict_vision_val_hand_aug.pkl'
file_path_hand_test = '/Users/rishabhdubey/Downloads/Project/Feature Vectors/17th/features_dict_vision_test_hand_aug.pkl'
file_path_hand_train = '/Users/rishabhdubey/Downloads/Project/Feature Vectors/17th/features_dict_vision_hand_aug.pkl'
input_folder = '/Users/rishabhdubey/Downloads/FracAtlas/images/BothComplete'
synthetic_folder = '/Users/rishabhdubey/Downloads/Project/Output_Synth'
masked_synthetic_folder = '/Users/rishabhdubey/Downloads/Project/Masked_Synth'
additional_folder = '/Users/rishabhdubey/Downloads/Project/Additional_Synth_new'
json_path = '/Users/rishabhdubey/Downloads/Project/coordinates_and_rotations.json'
original_folder = input_folder
file_path = '/Users/rishabhdubey/Downloads/FracAtlas/dataset.csv'

# Number of splits and folder 
max_nu_of_splits = 3
masked_synthetic_folder = additional_folder

In [ ]:
file_path_leg_validation,  file_path_leg_test, file_path_leg_train, file_path_hand_validation, file_path_hand_test, file_path_hand_train

---
Importing Libraries

---

In [2]:
import os
import random
import cv2
import numpy as np
import json
import pandas as pd
from tqdm.notebook import tqdm
from tqdm import tqdm
import warnings
import copy
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from itertools import permutations
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input, BatchNormalization, LeakyReLU, Conv1D, Flatten, Reshape, Lambda, Concatenate, MultiHeadAttention, LayerNormalization, Add, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
import keras_tuner as kt
import matplotlib.pyplot as plt
import pickle
import torch
from transformers import ViTImageProcessor, ViTModel
import cv2 as cv
from sklearn.model_selection import train_test_split
import torchvision.models as models
import torchvision.transforms as transforms
from torch import nn

# Install keras-tuner
!pip install keras-tuner --quiet



import warnings
warnings.filterwarnings('ignore')

# Open and read the JSON file this contains each 
with open(json_path, 'r') as file:
    coordinates_and_rotations = json.load(file)

---
# Creating X-ray splits
---
Creating augmented data. For each image 10 sets of labels (coordinates and rotatinos).

---

In [3]:
import os
import random
import cv2
import numpy as np
import json
from tqdm.notebook import tqdm

import warnings
warnings.filterwarnings('ignore')

# Constants
INITIAL_THRESHOLD = 220
MIN_THRESHOLD = 70
THRESHOLD_STEP = 10
MIN_INTENSITY_PERCENTAGE = 0.05
MAX_ATTEMPTS = 5
STANDARD_SIZE = (512, 512)
WINDOW_HEIGHT_RATIO_MIN = 0.9
WINDOW_HEIGHT_RATIO_MAX = 1.0
AUGMENTATION_COUNT = 10


# Create directories
os.makedirs(synthetic_folder, exist_ok=True)
os.makedirs(masked_synthetic_folder, exist_ok=True)
os.makedirs(additional_folder, exist_ok=True)

# to rotate the image by 90 degrees counterclockwise
def rotate_90_anticlockwise(image):
    return cv2.rotate(image, cv2.ROTATE_90_COUNTERCLOCKWISE)

# Function to crop edges
def crop_edges(image, threshold=160, crop_percent=0.05):
    h, w = image.shape[:2]
    crop_pixels = int(crop_percent * min(h, w))
    cropped_sides = []
    while True:
        top_edge = image[0, :]
        bottom_edge = image[-1, :]
        left_edge = image[:, 0]
        right_edge = image[:, -1]
        
        if np.all(top_edge > threshold):
            image = image[crop_pixels:, :]
            cropped_sides.append('top')
        if np.all(bottom_edge > threshold):
            image = image[:-crop_pixels, :]
            cropped_sides.append('bottom')
        if np.all(left_edge > threshold):
            image = image[:, crop_pixels:]
            cropped_sides.append('left')
        if np.all(right_edge > threshold):
            image = image[:, :-crop_pixels]
            cropped_sides.append('right')
        
        h, w = image.shape[:2]
        if crop_pixels >= h or crop_pixels >= w:
            break
        
        if not (np.all(top_edge > threshold) or np.all(bottom_edge > threshold) or np.all(left_edge > threshold) or np.all(right_edge > threshold)):
            break
    return image, cropped_sides

# to split and validate image
def split_and_validate(image, initial_threshold=INITIAL_THRESHOLD, min_intensity_percentage=MIN_INTENSITY_PERCENTAGE, max_attempts=MAX_ATTEMPTS):
    h, w = image.shape[:2]
    best_splits = None
    best_threshold = initial_threshold

    while initial_threshold >= MIN_THRESHOLD:
        for _ in range(max_attempts):
            x_coords = sorted([0, random.randint(w//3 - w//10, w//3 + w//10), random.randint(2*w//3 - w//10, 2*w//3 + w//10), w])
            split_images = [image[:, x_coords[i]:x_coords[i+1]] for i in range(3)]
            if all(np.sum(split >= initial_threshold) / split.size >= min_intensity_percentage for split in split_images):
                return split_images, x_coords, initial_threshold
        initial_threshold -= THRESHOLD_STEP

    # Fallback to variance-based splitting
    best_variance = float('inf')
    for _ in range(max_attempts):
        x_coords = sorted([0, random.randint(w//3 - w//10, w//3 + w//10), random.randint(2*w//3 - w//10, 2*w//3 + w//10), w])
        split_images = [image[:, x_coords[i]:x_coords[i+1]] for i in range(3)]
        variances = np.var([np.sum(split) for split in split_images])
        if variances < best_variance:
            best_variance = variances
            best_splits = split_images
            best_threshold = "fallback"

    return best_splits, x_coords, best_threshold

# to get maximum intensity window or the center adjusted for cropping
def max_intensity_window(split, window_height_ratio):
    h, w = split.shape[:2]
    window_height = int(h * window_height_ratio)
    crop_amount = h - window_height

    if random.choice([True, False]):
        # Crop from the top
        split = split[crop_amount:, :]
        y_center = (h // 2) + (crop_amount // 2)
    else:
        # Crop from the bottom
        split = split[:-crop_amount, :]
        y_center = (h // 2) - (crop_amount // 2)

    return y_center, split

# to rotate image randomly within +-10 degrees
def random_rotate(image):
    angle = random.uniform(-10, 10)
    h, w = image.shape[:2]
    center = (w // 2, h // 2)
    rotation_matrix = cv2.getRotationMatrix2D(center, angle, 1)
    rotated_image = cv2.warpAffine(image, rotation_matrix, (w, h))
    return rotated_image, angle

# to apply circular mask
def apply_circular_mask(image):
    h, w = image.shape[:2]
    mask = np.zeros((h, w), dtype=np.uint8)
    center = (w // 2, h // 2)
    radius = min(center) - 10
    cv2.circle(mask, center, radius, 255, -1)
    masked_image = cv2.bitwise_and(image, image, mask=mask)
    return masked_image

# Main processing
coordinates_and_rotations = []
for filename in tqdm(os.listdir(input_folder)):
    if filename.endswith('.png') or filename.endswith('.jpg'):
        image_path = os.path.join(input_folder, filename)
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

        # Rotate image by 90 degrees counterclockwise
        image = rotate_90_anticlockwise(image)
        
        # Step 1: Crop edges
        image, cropped_sides = crop_edges(image)
        
        for aug_num in range(1, AUGMENTATION_COUNT + 1):
            aug_filename_prefix = f"{aug_num}_"
            augmented_filename = aug_filename_prefix + filename
            
            # Step 2: Resize to 512x512
            resized_image = cv2.resize(image, STANDARD_SIZE)
            
            # Step 3 and 4: Split and validate
            splits, x_coords, used_threshold = split_and_validate(resized_image)
            
            split_data = {'filename': augmented_filename, 'splits': [], 'used_threshold': used_threshold}
            for split_idx, split_image in enumerate(splits):
                # Step 5: get y-coordinates
                window_height_ratio = random.uniform(WINDOW_HEIGHT_RATIO_MIN, WINDOW_HEIGHT_RATIO_MAX)
                y_center, split_image = max_intensity_window(split_image, window_height_ratio)
                split_data['splits'].append({'x_center': (x_coords[split_idx] + x_coords[split_idx+1]) // 2, 'y_center': y_center})
                
                # Step 6: Rotate and save images
                rotated_image, angle = random_rotate(split_image)
                additional_filename = f"{os.path.splitext(augmented_filename)[0]}_{split_idx+1}.jpg"
                additional_image_path = os.path.join(additional_folder, additional_filename)
                cv2.imwrite(additional_image_path, rotated_image)
                
                # Step 7: Resize back to 512x512
                split_image_resized = cv2.resize(split_image, STANDARD_SIZE)
                output_filename = f"{os.path.splitext(augmented_filename)[0]}_{split_idx+1}.jpg"
                output_image_path = os.path.join(synthetic_folder, output_filename)
                cv2.imwrite(output_image_path, split_image_resized)
                
                # Step 8: Apply circular mask
                masked_image = apply_circular_mask(split_image_resized)
                masked_image_path = os.path.join(masked_synthetic_folder, output_filename)
                cv2.imwrite(masked_image_path, masked_image)
                
                # data for JSON
                split_data['splits'][split_idx]['rotation'] = angle
            
            coordinates_and_rotations.append(split_data)

# Step 9: Saving coordinates and rotations to JSON
with open(json_path, 'w') as json_file:
    json.dump(coordinates_and_rotations, json_file, indent=4)

print("Processing complete.")


  0%|          | 0/3800 [00:00<?, ?it/s]

Premature end of JPEG file
Premature end of JPEG file
Premature end of JPEG file
Premature end of JPEG file
Premature end of JPEG file
Premature end of JPEG file
Premature end of JPEG file
Premature end of JPEG file
Premature end of JPEG file
Premature end of JPEG file
Premature end of JPEG file
Premature end of JPEG file
Premature end of JPEG file
Premature end of JPEG file
Premature end of JPEG file
Premature end of JPEG file
Premature end of JPEG file
Premature end of JPEG file
Premature end of JPEG file
Premature end of JPEG file
Premature end of JPEG file
Premature end of JPEG file
Premature end of JPEG file
Premature end of JPEG file
Premature end of JPEG file
Premature end of JPEG file
Premature end of JPEG file
Premature end of JPEG file
Premature end of JPEG file
Premature end of JPEG file
Premature end of JPEG file
Premature end of JPEG file
Premature end of JPEG file
Premature end of JPEG file
Premature end of JPEG file
Premature end of JPEG file
Premature end of JPEG file
P

Processing complete.


In [4]:
len(coordinates_and_rotations)

38000

In [5]:
# Initialize the coordinates dict to store the results
coordinates_d = {}

# Populate the coordinates
for item in coordinates_and_rotations:
    filename = item['filename']
    # Extract the (x, y) coordinates and convert them to integers
    coordinates = [(int(split['x_center']), int(split['y_center'])) for split in item['splits']]
    coordinates_d[filename] = coordinates
    
    
    
images = []
for i in coordinates_d.keys():
    images.append(i)


# Initialize the rotations dict to store the results
rotations_d = {}

# Populate the coordinates
for item in coordinates_and_rotations:
    filename = item['filename']
    # Extract coordinates and convert them to integers
    rotations = [int(split['rotation']) for split in item['splits']]
    rotations_d[filename] = rotations

len(rotations_d)

38000

In [6]:
img = [image.split('_',1)[1] for image in images]
img = set(img)
len(img)

3800

---
### Filtering dataset to Leg and Hand X-Rays ( Using the Dataset.csv file from the Fracatlas folder, one hot Labeled for image type)
---

In [7]:
# Read CSV file
df = pd.read_csv(file_path)
df.head()
columns = df.columns 
columns

Index(['image_id', 'hand', 'leg', 'hip', 'shoulder', 'mixed', 'hardware',
       'multiscan', 'fractured', 'fracture_count', 'frontal', 'lateral',
       'oblique'],
      dtype='object')

In [8]:
lists = {col: df[df[col] == 1]['image_id'].tolist() for col in columns}
img_hand = list( (set(lists['hand'])))
img_leg = list( (set(lists['leg'])))

len(img_hand)

1538

In [9]:
len(img_leg)

2273

Pre-Trained Feature Vector extraction model using VitImageProcessor using Vision Transformer

In [10]:
# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load the Vision Transformer model and feature extractor
feature_extractor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224-in21k')
model = ViTModel.from_pretrained('google/vit-base-patch16-224-in21k')

# Move model to GPU if available
if device.type == 'cuda':
    model.to(device)

def preprocess_image(image):
    image = cv2.resize(image, (224, 224))  # for Vision Transformer as it 224x224 input size
    image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)  # Converting grayscale to RGB
    pixel_values = feature_extractor(images=image, return_tensors="pt").pixel_values
    # Move tensor to GPU if available
    if device.type == 'cuda':
        pixel_values = pixel_values.to(device)
    return pixel_values

def extract_features(model, image):
    preprocessed_image = preprocess_image(image)
    with torch.no_grad():
        outputs = model(preprocessed_image)
        # Use the output of the last hidden state
        features = outputs.last_hidden_state[:, 0, :]  # [CLS] token representation
    # Move the result back to CPU
    return features.cpu().numpy().flatten()

Using device: cpu


Pre-Trained Feature Vector extraction model using Inception model, Inception version V3

In [11]:
# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load the pre-trained InceptionV3 model with aux_logits=True
model = models.inception_v3(pretrained=True, aux_logits=True)
model.eval()  # Set to evaluation mode

# Modify the model to output 2048-dimensional features by removing the classification head
model.fc = nn.Identity()  # Removing the final classification layer to get features before classification

# Move model to GPU if available
model = model.to(device)

# the preprocessing function
preprocess = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((299, 299)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

def preprocess_image(image):
    # Convert grayscale to RGB
    image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
    image = preprocess(image)
    image = image.unsqueeze(0)  # to Add batch dimension
    image = image.to(device)
    return image

def extract_features(model, image):
    preprocessed_image = preprocess_image(image)
    with torch.no_grad():
        outputs = model(preprocessed_image)
        # If aux_logits are enabled, the model will outputs a tuple
        if isinstance(outputs, tuple):
            features = outputs[0]  # the main output, i am ignoring auxiliary outputs
        else:
            features = outputs
    return features.cpu().numpy().flatten()


Using device: cpu


---
### Train Test Validation split for Leg and creating feature vectors for the Leg data
---

In [12]:
training, temping = train_test_split(img_leg, test_size=0.05, random_state=42)
testing, validating = train_test_split(temping, test_size=0.5, random_state=42)
# Check the size of each set
print(f"Train set size: {len(training)}")
print(f"Test set size: {len(testing)}")
print(f"Validation set size: {len(validating)}")

Train set size: 2159
Test set size: 57
Validation set size: 57


In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


In [14]:
features_dict_vision_test = {}

file_list = [filename for filename in os.listdir(original_folder) if (filename.endswith('.png') or filename.endswith('.jpg')) and filename in testing]

# Loop for each original image
for filename in tqdm(file_list, desc="Extracting test features"):
    # Read split images
    base_name, _ = os.path.splitext(filename)
    split_images = []
    split_features = []
    for i in range(1, max_nu_of_splits+1):
        split_image_path = os.path.join(masked_synthetic_folder, f"6_{base_name}_{i}.jpg")
        if os.path.exists(split_image_path):
            split_image = cv2.imread(split_image_path, cv2.IMREAD_GRAYSCALE)
            if split_image is None:
                print(f"Error: {split_image_path}")
                continue
            split_images.append(split_image)
            # features extracted for split image
            split_features.append(extract_features(model, split_image))
        else:
            break

    # features
    features_dict_vision_test[filename] = split_features
    

Extracting test features:   0%|          | 0/57 [00:00<?, ?it/s]

In [15]:
from tqdm import tqdm

features_dict_vision_test_inception = {}

# List of image files to process (from the testing set)
file_list = [filename for filename in os.listdir(original_folder) if (filename.endswith('.png') or filename.endswith('.jpg')) and filename in testing]

# Loop for each original image
for filename in tqdm(file_list, desc="Extracting test features with InceptionV3"):
    base_name, _ = os.path.splitext(filename)
    split_images = []
    split_features = []
    
    for i in range(1, max_nu_of_splits + 1):
        split_image_path = os.path.join(masked_synthetic_folder, f"6_{base_name}_{i}.jpg")
        if os.path.exists(split_image_path):
            split_image = cv2.imread(split_image_path, cv2.IMREAD_GRAYSCALE)
            if split_image is None:
                print(f"Error reading {split_image_path}")
                continue
            split_images.append(split_image)
            # Extract features for the split image
            split_features.append(extract_features(model, split_image))
        else:
            break
    
    # Store features in dict
    features_dict_vision_test_inception[filename] = split_features


Extracting test features with InceptionV3: 100%|█| 57/57 [00:10<00:00,  5.33it/s


In [16]:
features_dict_vision_val = {}

file_list = [filename for filename in os.listdir(original_folder) if (filename.endswith('.png') or filename.endswith('.jpg')) and filename in validating]

# Loop for each original image
for filename in tqdm(file_list, desc="Extracting validation features"):
    # Read split images
    base_name, _ = os.path.splitext(filename)
    split_images = []
    split_features = []
    for i in range(1, max_nu_of_splits+1):
        split_image_path = os.path.join(masked_synthetic_folder, f"6_{base_name}_{i}.jpg")
        if os.path.exists(split_image_path):
            split_image = cv2.imread(split_image_path, cv2.IMREAD_GRAYSCALE)
            if split_image is None:
                print(f"Error: {split_image_path}")
                continue
            split_images.append(split_image)
            # features extracted for split image
            split_features.append(extract_features(model, split_image))
        else:
            break

    # features 
    features_dict_vision_val[filename] = split_features
    

Extracting validation features: 100%|███████████| 57/57 [00:10<00:00,  5.36it/s]


In [17]:
features_dict_vision_val_inception = {}

file_list_val = [filename for filename in os.listdir(original_folder) if (filename.endswith('.png') or filename.endswith('.jpg')) and filename in validating]

for filename in tqdm(file_list_val, desc="Extracting validation features with InceptionV3"):
    base_name, _ = os.path.splitext(filename)
    split_images = []
    split_features = []
    for i in range(1, max_nu_of_splits+1):
        split_image_path = os.path.join(masked_synthetic_folder, f"6_{base_name}_{i}.jpg")
        if os.path.exists(split_image_path):
            split_image = cv2.imread(split_image_path, cv2.IMREAD_GRAYSCALE)
            if split_image is None:
                print(f"Error: {split_image_path}")
                continue
            split_images.append(split_image)
            split_features.append(extract_features(model, split_image))
        else:
            break

    features_dict_vision_val_inception[filename] = split_features

Extracting validation features with InceptionV3: 100%|█| 57/57 [00:08<00:00,  6.


In [18]:
len(training)

2159

In [19]:
file_list = [filename for filename in os.listdir(original_folder) if (filename.endswith('.png') or filename.endswith('.jpg')) and filename in training]
new_file_list = []

# Outer loop for each file in the original file_list
for file in file_list:
    # Inner loop to add prefixes from '1_' to '10_'
    for i in range(1, 11):
        new_file_list.append(f"{i}_{file}")
file_list = new_file_list


In [20]:
features_dict_vision = {}

# Loop for each original image
for filename in tqdm(file_list, desc="Extracting training features"):
    # Read split images
    base_name, _ = os.path.splitext(filename)
    split_images = []
    split_features = []
    for i in range(1, max_nu_of_splits+1):
        split_image_path = os.path.join(masked_synthetic_folder, f"{base_name}_{i}.jpg")
        if os.path.exists(split_image_path):
            split_image = cv2.imread(split_image_path, cv2.IMREAD_GRAYSCALE)
            if split_image is None:
                print(f"Error: {split_image_path}")
                continue
            split_images.append(split_image)
            # features extracted for split image
            split_features.append(extract_features(model, split_image))
        else:
            break

    # features in dict
    features_dict_vision[filename] = split_features
    

Extracting training features: 100%|███████| 21590/21590 [54:32<00:00,  6.60it/s]


In [21]:
features_dict_vision_inception = {}

for filename in tqdm(file_list, desc="Extracting validation features with InceptionV3"):
    base_name, _ = os.path.splitext(filename)
    split_images = []
    split_features = []
    for i in range(1, max_nu_of_splits+1):
        split_image_path = os.path.join(masked_synthetic_folder, f"6_{base_name}_{i}.jpg")
        if os.path.exists(split_image_path):
            split_image = cv2.imread(split_image_path, cv2.IMREAD_GRAYSCALE)
            if split_image is None:
                print(f"Error: {split_image_path}")
                continue
            split_images.append(split_image)
            split_features.append(extract_features(model, split_image))
        else:
            break

    features_dict_vision_inception[filename] = split_features
    

Extracting validation features with InceptionV3: 100%|█| 21590/21590 [00:00<00:0


Writing the Feature vectors to the Drive as a .pickle file, as they maintatin the structure when will be used in other notebook for Leg models Training

In [23]:
# Write the Leg feature vectors to a pickle file
with open(file_path_leg_validation, 'wb') as pickle_file:
    pickle.dump(features_dict_vision_val, pickle_file)

print(f"Leg Validation feature vectors saved to {file_path_leg_validation}")


with open(file_path_leg_test, 'wb') as pickle_file:
    pickle.dump(features_dict_vision_test, pickle_file)

print(f"Leg Test feature vectors saved to {file_path_leg_test}")

with open(file_path_leg_train, 'wb') as pickle_file:
    pickle.dump(features_dict_vision, pickle_file)

print(f"Leg Training Feature vectors saved to {file_path_leg_train}")


Leg Validation feature vectors saved to /Users/rishabhdubey/Downloads/Project/Feature Vectors/17th/features_dict_vision_val_leg_aug.pkl
Leg Test feature vectors saved to /Users/rishabhdubey/Downloads/Project/Feature Vectors/17th/features_dict_vision_test_leg_aug.pkl
Leg Training Feature vectors saved to /Users/rishabhdubey/Downloads/Project/Feature Vectors/17th/features_dict_vision_leg_aug.pkl


---
### Train Test Validation split for Leg and creating feature vectors for the Hand data
---

In [24]:
training, temping = train_test_split(img_hand, test_size=0.05, random_state=42)
testing, validating = train_test_split(temping, test_size=0.5, random_state=42)
# size check of each set
print(f"Train set size: {len(training)}")
print(f"Test set size: {len(testing)}")
print(f"Validation set size: {len(validating)}")

Train set size: 1461
Test set size: 38
Validation set size: 39


In [25]:
features_dict_vision_test = {}

file_list = [filename for filename in os.listdir(original_folder) if (filename.endswith('.png') or filename.endswith('.jpg')) and filename in testing]

# Loop for each original image
for filename in tqdm(file_list, desc="Extracting test features with Vision Transformer"):
    # Read split images
    base_name, _ = os.path.splitext(filename)
    split_images = []
    split_features = []
    for i in range(1, max_nu_of_splits+1):
        split_image_path = os.path.join(masked_synthetic_folder, f"6_{base_name}_{i}.jpg")
        if os.path.exists(split_image_path):
            split_image = cv2.imread(split_image_path, cv2.IMREAD_GRAYSCALE)
            if split_image is None:
                print(f"Error: {split_image_path}")
                continue
            split_images.append(split_image)
            # features extracted for split image
            split_features.append(extract_features(model, split_image))
        else:
            break

    # features in dict
    features_dict_vision_test[filename] = split_features
    

Extracting test features with Vision Transformer: 100%|█| 38/38 [00:05<00:00,  7


In [26]:
from tqdm import tqdm

features_dict_vision_test_inception = {}

# List of image files to process (from the testing set)
file_list = [filename for filename in os.listdir(original_folder) if (filename.endswith('.png') or filename.endswith('.jpg')) and filename in testing]

# Loop for each original image
for filename in tqdm(file_list, desc="Extracting test features with InceptionV3"):
    base_name, _ = os.path.splitext(filename)
    split_images = []
    split_features = []
    
    for i in range(1, max_nu_of_splits + 1):
        split_image_path = os.path.join(masked_synthetic_folder, f"6_{base_name}_{i}.jpg")
        if os.path.exists(split_image_path):
            split_image = cv2.imread(split_image_path, cv2.IMREAD_GRAYSCALE)
            if split_image is None:
                print(f"Error reading {split_image_path}")
                continue
            split_images.append(split_image)
            # Extract features for the split image
            split_features.append(extract_features(model, split_image))
        else:
            break
    
    # Store features in dictionary
    features_dict_vision_test_inception[filename] = split_features


Extracting test features with InceptionV3: 100%|█| 38/38 [00:04<00:00,  7.62it/s


In [27]:
features_dict_vision_val = {}

file_list = [filename for filename in os.listdir(original_folder) if (filename.endswith('.png') or filename.endswith('.jpg')) and filename in validating]

# Loop for each original image
for filename in tqdm(file_list, desc="Extracting validation features with Vision Transformer"):
    # Read split images
    base_name, _ = os.path.splitext(filename)
    split_images = []
    split_features = []
    for i in range(1, max_nu_of_splits+1):
        split_image_path = os.path.join(masked_synthetic_folder, f"6_{base_name}_{i}.jpg")
        if os.path.exists(split_image_path):
            split_image = cv2.imread(split_image_path, cv2.IMREAD_GRAYSCALE)
            if split_image is None:
                print(f"Error: {split_image_path}")
                continue
            split_images.append(split_image)
            # features extracted for split image
            split_features.append(extract_features(model, split_image))
        else:
            break

    # features in dict
    features_dict_vision_val[filename] = split_features
    

Extracting validation features with Vision Transformer: 100%|█| 39/39 [00:05<00:


In [28]:
features_dict_vision_val_inception = {}

file_list_val = [filename for filename in os.listdir(original_folder) if (filename.endswith('.png') or filename.endswith('.jpg')) and filename in validating]

for filename in tqdm(file_list_val, desc="Extracting validation features with InceptionV3"):
    base_name, _ = os.path.splitext(filename)
    split_images = []
    split_features = []
    for i in range(1, max_nu_of_splits+1):
        split_image_path = os.path.join(masked_synthetic_folder, f"6_{base_name}_{i}.jpg")
        if os.path.exists(split_image_path):
            split_image = cv2.imread(split_image_path, cv2.IMREAD_GRAYSCALE)
            if split_image is None:
                print(f"Error: {split_image_path}")
                continue
            split_images.append(split_image)
            split_features.append(extract_features(model, split_image))
        else:
            break

    features_dict_vision_val_inception[filename] = split_features

Extracting validation features with InceptionV3: 100%|█| 39/39 [00:05<00:00,  7.


In [29]:
file_list = [filename for filename in os.listdir(original_folder) if (filename.endswith('.png') or filename.endswith('.jpg')) and filename in training]
new_file_list = []

# Outer loop for each file in the original file_list
for file in file_list:
    # Inner loop to add prefixes from '1_' to '10_'
    for i in range(1, 11):
        new_file_list.append(f"{i}_{file}")
file_list = new_file_list


In [30]:
len(file_list)

14580

In [31]:
features_dict_vision = {}
# Loop for each original imaget
for filename in tqdm(file_list, desc="Extracting training features with Vision Transformer"):
    # Read split images
    base_name, _ = os.path.splitext(filename)
    split_images = []
    split_features = []
    for i in range(1, max_nu_of_splits+1):
        split_image_path = os.path.join(masked_synthetic_folder, f"{base_name}_{i}.jpg")
        if os.path.exists(split_image_path):
            split_image = cv2.imread(split_image_path, cv2.IMREAD_GRAYSCALE)
            if split_image is None:
                print(f"Error: {split_image_path}")
                continue
            split_images.append(split_image)
            # features extracted for split image
            split_features.append(extract_features(model, split_image))
        else:
            break

    # features
    features_dict_vision[filename] = split_features
    

Extracting training features with Vision Transformer: 100%|█| 14580/14580 [40:24


In [32]:
features_dict_vision_inception = {}

for filename in tqdm(file_list, desc="Extracting validation features with InceptionV3"):
    base_name, _ = os.path.splitext(filename)
    split_images = []
    split_features = []
    for i in range(1, max_nu_of_splits+1):
        split_image_path = os.path.join(masked_synthetic_folder, f"6_{base_name}_{i}.jpg")
        if os.path.exists(split_image_path):
            split_image = cv2.imread(split_image_path, cv2.IMREAD_GRAYSCALE)
            if split_image is None:
                print(f"Error: {split_image_path}")
                continue
            split_images.append(split_image)
            split_features.append(extract_features(model, split_image))
        else:
            break

    features_dict_vision_inception[filename] = split_features
    

Extracting validation features with InceptionV3: 100%|█| 14580/14580 [00:00<00:0


---
Writing the Feature vectors to the Drive as a .pickle file, as they maintatin the structure when will be used in other notebook for Leg models Training

---

In [33]:
# Write the hand feature vectors to a pickle file
with open(file_path_hand_validation, 'wb') as pickle_file:
    pickle.dump(features_dict_vision_val, pickle_file)

print(f"Hand Validation Feature vectors  saved to {file_path_hand_validation}")

with open(file_path_hand_test, 'wb') as pickle_file:
    pickle.dump(features_dict_vision_test, pickle_file)

print(f"Hand Test Feature vectors  saved to {file_path_hand_test}")


with open(file_path_hand_train, 'wb') as pickle_file:
    pickle.dump(features_dict_vision, pickle_file)

print(f"Hand Training Feature vectors saved to {file_path_hand_train}")

Hand Validation Feature vectors  saved to /Users/rishabhdubey/Downloads/Project/Feature Vectors/17th/features_dict_vision_val_hand_aug.pkl
Hand Test Feature vectors  saved to /Users/rishabhdubey/Downloads/Project/Feature Vectors/17th/features_dict_vision_test_hand_aug.pkl
Hand Training Feature vectors saved to /Users/rishabhdubey/Downloads/Project/Feature Vectors/17th/features_dict_vision_hand_aug.pkl


---
Writing the Json Object Containing Labels for the Image Data to Drive


In [34]:
with open(json_path, 'w') as file:
    json.dump(coordinates_and_rotations, file, indent=4) 

---